In [1]:
import os
import pandas as pd
import glob
import pickle
from selenium import webdriver
from selenium.webdriver.common.by import By
import concurrent.futures 
from pycaret.classification import *
from sklearn.metrics import accuracy_score
import requests
from bs4 import BeautifulSoup
import time
import random
from fuzzywuzzy import fuzz, process


C:\Users\idotz\anaconda3\envs\p37\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\idotz\anaconda3\envs\p37\lib\site-packages\numpy\.libs\libopenblas.4SP5SUA7CBGXUEOC35YP2ASOICYYEQZZ.gfortran-win_amd64.dll
C:\Users\idotz\anaconda3\envs\p37\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  stacklevel=1)


In [2]:
open_file = open('model_columns.pkl', "rb")
model_columns = list(pickle.load(open_file))
open_file.close()

In [3]:
scraped_cols = ['Total shots','Expected goals (xG)','Offsides','Accurate passes percentage','score','Corners','Fouls committed','Shots on target','Blocks']
pred_cols = dict(zip(model_columns, scraped_cols))
model = load_model('lr_model')

Transformation Pipeline and Model Successfully Loaded


In [154]:
pred_cols

{'Shot Attempts': 'Total shots',
 'Total xG': 'Expected goals (xG)',
 'Offsides': 'Offsides',
 'Pass Completion': 'Accurate passes percentage',
 'Goals': 'score',
 'Corners': 'Corners',
 'Fouls Committed': 'Fouls committed',
 'Total Shots on Target': 'Shots on target',
 'Shots Blocked': 'Blocks'}

In [4]:
from os import walk

filenames = next(walk('C:/Users\idotz/OneDrive - Northeastern University/Python/Soccer Analysis'), (None, None, []))[2]
filenames = [f for f in filenames if '2022' in f]

In [5]:
filenames

['20220826_data.pkl',
 '20220828_data.pkl',
 '20220829_data.pkl',
 '20220830_data.pkl',
 '20220831_data.pkl',
 '20220901_data.pkl',
 '20220902_data.pkl',
 '20220903_data.pkl',
 '20220904_data.pkl']

In [27]:
all_data = []
for f in filenames:
    open_file = open(f, "rb")
    d = pickle.load(open_file)
    all_data.append(d)
    open_file.close()
    
all_data = [i for j in all_data for i in j]

In [28]:
f = []
for dic in all_data:
    d = {}
    for k, v in dic.items():
        try:
            d[k] = float(v[0]) - float(v[1])
        except:
            d[k] = v
    f.append(d)


In [41]:
f

[{'Expected goals (xG)': 0.23,
  'Total shots': 3.0,
  'Big chances': 1.0,
  'Big chances missed': 1.0,
  'Accurate passes': 7.0,
  'Accurate passes percentage': 0.04999999999999993,
  'Fouls committed': 2.0,
  'Offsides': 0.0,
  'Corners': -2.0,
  'Shots off target': 1.0,
  'Shots on target': 2.0,
  'Blocked shots': 0.0,
  'Hit woodwork': 0.0,
  'Shots inside box': 0.0,
  'Shots outside box': 3.0,
  'xG open play': 0.15000000000000002,
  'xG set play': -0.05,
  'xG on target (xGOT)': 0.92,
  'Passes': 0.0,
  'Own half': -14.0,
  'Opposition half': 21.0,
  'Accurate long balls': 12.0,
  'Accurate long balls percentage': 0.43999999999999995,
  'Accurate crosses': 2.0,
  'Accurate crosses percentage': 0.22999999999999998,
  'Throws': 0.0,
  'Yellow cards': 0.0,
  'Red cards': 0.0,
  'Tackles won': 2.0,
  'Tackles won percentage': 0.36,
  'Interceptions': 4.0,
  'Blocks': 0.0,
  'Clearances': 0.0,
  'Keeper saves': -1.0,
  'Duels won': 3.0,
  'Ground duels won': -3.0,
  'Ground duels won 

In [66]:
df = pd.DataFrame(f)
df = df[df['half_time'] == 'Half-Time']
df = df[(df['odds'] != '') & (df['odds'] != {})]

In [67]:
df = df.reset_index(drop = True)

In [68]:
df2 = pd.json_normalize(df['bet'])
df = pd.concat([df, df2], axis = 1)

In [69]:
df['o_names'] = df['odds_url'].apply(lambda x: ['draw'] + x.split('/')[-2].split('-v-'))

In [70]:
r = []
for i in df[['odds','o_names']].to_numpy():
    oo = [getname(name, list(i[0].keys())) for name in i[1]]
    oo = [i[0][key] for key in oo]
    r.append(oo)
df['oo'] = r

In [71]:
split_df = pd.DataFrame(df['oo'].tolist(), columns=['0', '1', '2'])
df = pd.concat([df, split_df], axis=1)

In [75]:
df

,Expected goals (xG),Total shots,Big chances,Big chances missed,Accurate passes,Accurate passes percentage,Fouls committed,Offsides,Corners,Shots off target,Shots on target,Blocked shots,Hit woodwork,Shots inside box,Shots outside box,xG open play,xG set play,xG on target (xGOT),Passes,Own half,Opposition half,Accurate long balls,Accurate long balls percentage,Accurate crosses,Accurate crosses percentage,Throws,Yellow cards,Red cards,Tackles won,Tackles won percentage,Interceptions,Blocks,Clearances,Keeper saves,Duels won,Ground duels won,Ground duels won percentage,Aerial duels won,Aerial duels won percentage,Successful dribbles,Successful dribbles percentage,score,posession,match_name,half_time,url,odds_url,odds,full_odds,bet,xG penalty,draw,home,away,odds_checker_sum,winner,predicted_probs,o_names,oo,0,1,2
0,0.89,7.0,1.0,0.0,77.0,0.06,-3.0,1.0,1.0,4.0,0.0,3.0,0.0,7.0,0.0,0.97,-0.08,0.28,77.0,43.0,34.0,8.0,0.13,3.0,0.23,2.0,-2.0,0.0,-3.0,0.26,2.0,-3.0,-1.0,0.0,-2.0,-1.0,-0.03,-1.0,-0.10,4.0,-0.31,0.0,20.0,puebla-vs-fc-juarez,Half-Time,https://www.fotmob.com/match/3888550/matchfact...,https://www.oddschecker.com/football/world/mex...,"{'Draw': 2.3, 'Puebla': 2.3875, 'FC Juarez': 4.2}","[[2.4, 2.375, 2.3, 2.375, 2.3, 2.3, 2.2, 2.5, ...","{'draw': 'no value', 'home': 0.07935, 'away': ...",NaN,no value,0.07935,no value,1.091726,1,"[0.331, 0.511, 0.158]","[draw, puebla, fc-juarez]","[2.3, 2.3875, 4.2]",2.300000,2.387500,4.200000
1,-1.21,-5.0,-1.0,0.0,-54.0,-0.09,9.0,-1.0,-1.0,-1.0,-2.0,-2.0,0.0,-4.0,-1.0,-0.96,-0.26,-1.28,-49.0,5.0,-59.0,-7.0,-0.15,-1.0,0.00,-3.0,2.0,0.0,5.0,0.40,0.0,2.0,9.0,0.0,-3.0,-5.0,-0.16,2.0,0.06,-2.0,-0.11,-1.0,-16.0,mazatlan-fc-vs-cf-america,Half-Time,https://www.fotmob.com/match/3888551/matchfact...,https://www.oddschecker.com/football/world/mex...,"{'CF America': 1.25, 'Draw': 4.75, 'Mazatlan F...","[[1.2857142857142856, 1.25, 1.2222222222222223...","{'draw': 'no value', 'home': 'no value', 'away...",NaN,no value,no value,0.3375,1.087449,2,"[0.057, 0.008, 0.935]","[draw, mazatlan-fc, cf-america]","[4.75, 13.0, 1.25]",4.750000,13.000000,1.250000
2,-0.74,-3.0,-1.0,0.0,-140.0,-0.11,-3.0,0.0,-1.0,0.0,-1.0,-2.0,0.0,-1.0,-2.0,-0.06,-0.67,-1.25,-141.0,-31.0,-109.0,-7.0,-0.30,-1.0,-0.01,-5.0,-2.0,0.0,-2.0,0.43,4.0,1.0,5.0,-1.0,-15.0,-8.0,-0.40,-7.0,-0.64,-4.0,-0.80,-2.0,-26.0,fc-utrecht-vs-ajax,Half-Time,https://www.fotmob.com/match/3900373/matchfact...,https://www.oddschecker.com/football/netherlan...,"{'Ajax': 1.024390243902439, 'Draw': 20.0, 'FC ...","[[1.025, 1.0151515151515151, 1.02, 1.02, 1.02,...","{'draw': 0.02265, 'home': 'no value', 'away': ...",NaN,0.02265,no value,no value,1.044048,2,"[0.093, 0.015, 0.893]","[draw, fc-utrecht, ajax]","[20.0, 56.0, 1.024390243902439]",20.000000,56.000000,1.024390
3,0.37,4.0,1.0,1.0,56.0,0.04,-1.0,-1.0,-1.0,3.0,2.0,-1.0,1.0,2.0,2.0,0.37,NaN,0.97,56.0,-14.0,70.0,-4.0,-0.01,4.0,0.40,-2.0,-2.0,0.0,-3.0,-0.17,-4.0,1.0,-6.0,-3.0,-5.0,1.0,0.02,-6.0,-0.36,3.0,0.04,-1.0,12.0,nantes-vs-toulouse,Half-Time,https://www.fotmob.com/match/3904422/matchfact...,https://www.oddschecker.com/football/france/li...,"{'Toulouse': 1.6, 'Draw': 3.5, 'Nantes': 6.5}","[[1.6153846153846154, 1.5714285714285714, 1.57...","{'draw': 'no value', 'home': 0.0202, 'away': '...",NaN,no value,0.0202,no value,1.064560,2,"[0.271, 0.188, 0.541]","[draw, nantes, toulouse]","[3.5, 6.5, 1.6]",3.500000,6.500000,1.600000
4,-2.94,-12.0,-3.0,0.0,-242.0,-0.33,-1.0,-1.0,-4.0,0.0,-6.0,-6.0,1.0,-10.0,-2.0,-2.18,-0.76,-3.13,-234.0,-105.0,-137.0,-4.0,-0.31,-1.0,0.13,-8.0,1.0,0.0,1.0,0.07,3.0,6.0,13.0,2.0,7.0,5.0,0.24,2.0,0.10,3.0,0.60,-4.0,-50.0,dundee-united-vs-celtic,Half-Time,https://www.fotmob.com/match/3902828/matchfact...,https://www.oddschecker.com/football/scottish/...,"{'Celtic': 1.04, 'Dundee Utd': 201.0, 'Draw': ...","[[1.0002, 1.0909090909090908, 1.04, 1.0, 1.05,...","{'draw': 'no value', 'home': 'no value', 'away...",NaN,no value,no value,0.474,0.970938,2,"[0.002, 0.0, 0.998]","[draw, dundee-utd, celtic]","

In [11]:
def getname(name, l):
    if len(l) == 0:
        return ''
    highest = process.extractOne(name,l)
    return highest[0]

In [12]:
def score(url):
    driver = webdriver.Chrome()
    driver.implicitly_wait(1)
    driver.get(url)
    score = driver.find_element(By.XPATH, '//span[@class="css-bw7eig-topRow"]').get_attribute('textContent').split(' ')
    score = int(score[0]) - int(score[2])
    driver.quit()
    return score

In [13]:
def get_winner(url):
    page = requests.get(url)
    time.sleep(random.randint(0, 4))
    soup = BeautifulSoup(page.content, 'html.parser')
    score = soup.find_all("span", {"class": "css-bw7eig-topRow"})[0].text.split(' - ')
    score = int(score[0]) - int(score[1])
    score = winner(score)
    return score

In [20]:
scores = []
for i in range(0, len(df['url']), 3):
    x = list(df['url'][i:i + 3])
    print(x)
    with concurrent.futures.ThreadPoolExecutor() as executor:
        r = executor.map(get_winner, x)
    for result in r:
        scores.append(result)

['https://www.fotmob.com/match/3888550/matchfacts/puebla-vs-fc-juarez', 'https://www.fotmob.com/match/3888551/matchfacts/mazatlan-fc-vs-cf-america', 'https://www.fotmob.com/match/3900373/matchfacts/fc-utrecht-vs-ajax']
['https://www.fotmob.com/match/3904422/matchfacts/nantes-vs-toulouse', 'https://www.fotmob.com/match/3902828/matchfacts/dundee-united-vs-celtic', 'https://www.fotmob.com/match/3916965/matchfacts/gent-vs-royal-antwerp']
['https://www.fotmob.com/match/3887414/matchfacts/randers-fc-vs-aalborg-bk', 'https://www.fotmob.com/match/3887416/matchfacts/viborg-vs-lyngby', 'https://www.fotmob.com/match/3905446/matchfacts/lugano-vs-st.-gallen']
['https://www.fotmob.com/match/3900374/matchfacts/excelsior-vs-psv-eindhoven', 'https://www.fotmob.com/match/3900375/matchfacts/fc-volendam-vs-fc-twente', 'https://www.fotmob.com/match/3900963/matchfacts/aston-villa-vs-west-ham-united']
['https://www.fotmob.com/match/3900971/matchfacts/wolverhampton-wanderers-vs-newcastle-united', 'https://www

['https://www.fotmob.com/match/3888623/matchfacts/necaxa-vs-leon', 'https://www.fotmob.com/match/3900986/matchfacts/everton-vs-liverpool', 'https://www.fotmob.com/match/3902849/matchfacts/celtic-vs-rangers']
['https://www.fotmob.com/match/3917967/matchfacts/mallorca-vs-girona', 'https://www.fotmob.com/match/3919120/matchfacts/fiorentina-vs-juventus', 'https://www.fotmob.com/match/3903581/matchfacts/bayer-leverkusen-vs-sc-freiburg']
['https://www.fotmob.com/match/3903586/matchfacts/vfl-bochum-vs-werder-bremen', 'https://www.fotmob.com/match/3903582/matchfacts/union-berlin-vs-bayern-m%C3%BCnchen', 'https://www.fotmob.com/match/3903585/matchfacts/wolfsburg-vs-1.-fc-k%C3%B6ln']
['https://www.fotmob.com/match/3900983/matchfacts/brentford-vs-leeds-united', 'https://www.fotmob.com/match/3900985/matchfacts/chelsea-vs-west-ham-united', 'https://www.fotmob.com/match/3900988/matchfacts/newcastle-united-vs-crystal-palace']
['https://www.fotmob.com/match/3900990/matchfacts/tottenham-hotspur-vs-fulh

In [19]:
def winner(x):
    if x == 0:
        return 0
    elif x > 0:
        return 1
    else:
        return 2

In [34]:
df['final_score'] = scores

In [35]:
df['odds_predict'] = df['oo'].apply(lambda x: np.argmin(x))

In [23]:
df

,Expected goals (xG),Total shots,Big chances,Big chances missed,Accurate passes,Accurate passes percentage,Fouls committed,Offsides,Corners,Shots off target,Shots on target,Blocked shots,Hit woodwork,Shots inside box,Shots outside box,xG open play,xG set play,xG on target (xGOT),Passes,Own half,Opposition half,Accurate long balls,Accurate long balls percentage,Accurate crosses,Accurate crosses percentage,Throws,Yellow cards,Red cards,Tackles won,Tackles won percentage,Interceptions,Blocks,Clearances,Keeper saves,Duels won,Ground duels won,Ground duels won percentage,Aerial duels won,Aerial duels won percentage,Successful dribbles,Successful dribbles percentage,score,posession,match_name,half_time,url,odds_url,odds,o_names,oo,0,1,2,final_score,odds_precict
0,0.89,7.0,1.0,0.0,77.0,0.06,-3.0,1.0,1.0,4.0,0.0,3.0,0.0,7.0,0.0,0.97,-0.08,0.28,77.0,43.0,34.0,8.0,0.13,3.0,0.23,2.0,-2.0,0.0,-3.0,0.26,2.0,-3.0,-1.0,0.0,-2.0,-1.0,-0.03,-1.0,-0.10,4.0,-0.31,0.0,20.0,puebla-vs-fc-juarez,Half-Time,https://www.fotmob.com/match/3888550/matchfact...,https://www.oddschecker.com/football/world/mex...,"{'Draw': 2.3, 'Puebla': 2.3875, 'FC Juarez': 4.2}","[draw, puebla, fc-juarez]","[2.3, 2.3875, 4.2]",2.300000,2.387500,4.200000,0,0
1,-1.21,-5.0,-1.0,0.0,-54.0,-0.09,9.0,-1.0,-1.0,-1.0,-2.0,-2.0,0.0,-4.0,-1.0,-0.96,-0.26,-1.28,-49.0,5.0,-59.0,-7.0,-0.15,-1.0,0.00,-3.0,2.0,0.0,5.0,0.40,0.0,2.0,9.0,0.0,-3.0,-5.0,-0.16,2.0,0.06,-2.0,-0.11,-1.0,-16.0,mazatlan-fc-vs-cf-america,Half-Time,https://www.fotmob.com/match/3888551/matchfact...,https://www.oddschecker.com/football/world/mex...,"{'CF America': 1.25, 'Draw': 4.75, 'Mazatlan F...","[draw, mazatlan-fc, cf-america]","[4.75, 13.0, 1.25]",4.750000,13.000000,1.250000,2,2
2,-0.74,-3.0,-1.0,0.0,-140.0,-0.11,-3.0,0.0,-1.0,0.0,-1.0,-2.0,0.0,-1.0,-2.0,-0.06,-0.67,-1.25,-141.0,-31.0,-109.0,-7.0,-0.30,-1.0,-0.01,-5.0,-2.0,0.0,-2.0,0.43,4.0,1.0,5.0,-1.0,-15.0,-8.0,-0.40,-7.0,-0.64,-4.0,-0.80,-2.0,-26.0,fc-utrecht-vs-ajax,Half-Time,https://www.fotmob.com/match/3900373/matchfact...,https://www.oddschecker.com/football/netherlan...,"{'Ajax': 1.024390243902439, 'Draw': 20.0, 'FC ...","[draw, fc-utrecht, ajax]","[20.0, 56.0, 1.024390243902439]",20.000000,56.000000,1.024390,2,2
3,0.37,4.0,1.0,1.0,56.0,0.04,-1.0,-1.0,-1.0,3.0,2.0,-1.0,1.0,2.0,2.0,0.37,NaN,0.97,56.0,-14.0,70.0,-4.0,-0.01,4.0,0.40,-2.0,-2.0,0.0,-3.0,-0.17,-4.0,1.0,-6.0,-3.0,-5.0,1.0,0.02,-6.0,-0.36,3.0,0.04,-1.0,12.0,nantes-vs-toulouse,Half-Time,https://www.fotmob.com/match/3904422/matchfact...,https://www.oddschecker.com/football/france/li...,"{'Toulouse': 1.6, 'Draw': 3.5, 'Nantes': 6.5}","[draw, nantes, toulouse]","[3.5, 6.5, 1.6]",3.500000,6.500000,1.600000,1,2
4,-2.94,-12.0,-3.0,0.0,-242.0,-0.33,-1.0,-1.0,-4.0,0.0,-6.0,-6.0,1.0,-10.0,-2.0,-2.18,-0.76,-3.13,-234.0,-105.0,-137.0,-4.0,-0.31,-1.0,0.13,-8.0,1.0,0.0,1.0,0.07,3.0,6.0,13.0,2.0,7.0,5.0,0.24,2.0,0.10,3.0,0.60,-4.0,-50.0,dundee-united-vs-celtic,Half-Time,https://www.fotmob.com/match/3902828/matchfact...,https://www.oddschecker.com/football/scottish/...,"{'Celtic': 1.04, 'Dundee Utd': 201.0, 'Draw': ...","[draw, dundee-utd, celtic]","[226.0, 201.0, 1.04]",226.000000,201.000000,1.040000,2,2
5,-0.95,1.0,-2.0,0.0,-21.0,-0.02,0.0,1.0,2.0,2.0,-1.0,0.0,0.0,0.0,1.0,-0.04,-0.13,-1.99,-19.0,-59.0,38.0,-1.0,0.20,0.0,-0.04,5.0,0.0,0.0,-4.0,-0.28,4.0,0.0,-4.0,0.0,-5.0,-6.0,-0.26,1.0,0.05,0.0,-0.25,-1.0,-4.0,gent-vs-royal-antwerp,Half-Time,https://www.fotmob.com/match/3916965/matchfact...,https://www.oddschecker.com/football/belgium/j...,"{'Draw': 3.126923076923077, 'Antwerp': 1.81666...","[draw, gent, antwerp]","[3.126923076923077, 4.5, 1.8166666666666669]",3.126923,4.500000,1.816667,2,2
6,-0.33,-3.0,1.0,0.0,-104.0,-0.15,-1.0,3.0,-3.0,0.0,0.0,-3.0,0.0,1.0,-4.0,0.31,-0.64,0.71,-97.0,-51.0,-53.0,3.0,-0.09,2.0,0.29,-1.0,0.0,0.0,2.0,-0.06,-1.0,3.0,6.0,1.0,7.0,3.0,0.10,4.0,0.25,-1.0,0.11,1.0,-22.0,randers-fc-vs-aalborg-bk,Half-Time,https://www.fotmob.com/match/3887414/matchfact...,https://www.oddschecker.com/foo

In [40]:
accuracy_score(df['final_score'], df['odds_predict'])

0.6683673469387755

In [36]:
df = df.drop(['match_name', 'half_time','url','odds_url','odds','o_names','oo'], axis = 1)

In [37]:
df.to_csv('data_0905.csv')

In [300]:
df

,Expected goals (xG),Total shots,Big chances,Big chances missed,Accurate passes,Accurate passes percentage,Fouls committed,Offsides,Corners,Shots off target,Shots on target,Blocked shots,Hit woodwork,Shots inside box,Shots outside box,xG open play,xG set play,xG on target (xGOT),Passes,Own half,Opposition half,Accurate long balls,Accurate long balls percentage,Accurate crosses,Accurate crosses percentage,Throws,Yellow cards,Red cards,Tackles won,Tackles won percentage,Interceptions,Blocks,Clearances,Keeper saves,Duels won,Ground duels won,Ground duels won percentage,Aerial duels won,Aerial duels won percentage,Successful dribbles,Successful dribbles percentage,score,posession,match_name,half_time,url,odds_url,odds,o_names,oo,0,1,2,final_score,odds_precict
0,0.89,7.0,1.0,0.0,77.0,0.06,-3.0,1.0,1.0,4.0,0.0,3.0,0.0,7.0,0.0,0.97,-0.08,0.28,77.0,43.0,34.0,8.0,0.13,3.0,0.23,2.0,-2.0,0.0,-3.0,0.26,2.0,-3.0,-1.0,0.0,-2.0,-1.0,-0.03,-1.0,-0.10,4.0,-0.31,0.0,20.0,puebla-vs-fc-juarez,Half-Time,https://www.fotmob.com/match/3888550/matchfact...,https://www.oddschecker.com/football/world/mex...,"{'Draw': 2.3, 'Puebla': 2.3875, 'FC Juarez': 4.2}","[draw, puebla, fc-juarez]","[2.3, 2.3875, 4.2]",2.300000,2.387500,4.200000,0,0
1,-1.21,-5.0,-1.0,0.0,-54.0,-0.09,9.0,-1.0,-1.0,-1.0,-2.0,-2.0,0.0,-4.0,-1.0,-0.96,-0.26,-1.28,-49.0,5.0,-59.0,-7.0,-0.15,-1.0,0.00,-3.0,2.0,0.0,5.0,0.40,0.0,2.0,9.0,0.0,-3.0,-5.0,-0.16,2.0,0.06,-2.0,-0.11,-1.0,-16.0,mazatlan-fc-vs-cf-america,Half-Time,https://www.fotmob.com/match/3888551/matchfact...,https://www.oddschecker.com/football/world/mex...,"{'CF America': 1.25, 'Draw': 4.75, 'Mazatlan F...","[draw, mazatlan-fc, cf-america]","[4.75, 13.0, 1.25]",4.750000,13.000000,1.250000,2,2
2,-0.74,-3.0,-1.0,0.0,-140.0,-0.11,-3.0,0.0,-1.0,0.0,-1.0,-2.0,0.0,-1.0,-2.0,-0.06,-0.67,-1.25,-141.0,-31.0,-109.0,-7.0,-0.30,-1.0,-0.01,-5.0,-2.0,0.0,-2.0,0.43,4.0,1.0,5.0,-1.0,-15.0,-8.0,-0.40,-7.0,-0.64,-4.0,-0.80,-2.0,-26.0,fc-utrecht-vs-ajax,Half-Time,https://www.fotmob.com/match/3900373/matchfact...,https://www.oddschecker.com/football/netherlan...,"{'Ajax': 1.024390243902439, 'Draw': 20.0, 'FC ...","[draw, fc-utrecht, ajax]","[20.0, 56.0, 1.024390243902439]",20.000000,56.000000,1.024390,2,2
3,0.37,4.0,1.0,1.0,56.0,0.04,-1.0,-1.0,-1.0,3.0,2.0,-1.0,1.0,2.0,2.0,0.37,NaN,0.97,56.0,-14.0,70.0,-4.0,-0.01,4.0,0.40,-2.0,-2.0,0.0,-3.0,-0.17,-4.0,1.0,-6.0,-3.0,-5.0,1.0,0.02,-6.0,-0.36,3.0,0.04,-1.0,12.0,nantes-vs-toulouse,Half-Time,https://www.fotmob.com/match/3904422/matchfact...,https://www.oddschecker.com/football/france/li...,"{'Toulouse': 1.6, 'Draw': 3.5, 'Nantes': 6.5}","[draw, nantes, toulouse]","[3.5, 6.5, 1.6]",3.500000,6.500000,1.600000,1,2
4,-2.94,-12.0,-3.0,0.0,-242.0,-0.33,-1.0,-1.0,-4.0,0.0,-6.0,-6.0,1.0,-10.0,-2.0,-2.18,-0.76,-3.13,-234.0,-105.0,-137.0,-4.0,-0.31,-1.0,0.13,-8.0,1.0,0.0,1.0,0.07,3.0,6.0,13.0,2.0,7.0,5.0,0.24,2.0,0.10,3.0,0.60,-4.0,-50.0,dundee-united-vs-celtic,Half-Time,https://www.fotmob.com/match/3902828/matchfact...,https://www.oddschecker.com/football/scottish/...,"{'Celtic': 1.04, 'Dundee Utd': 201.0, 'Draw': ...","[draw, dundee-utd, celtic]","[226.0, 201.0, 1.04]",226.000000,201.000000,1.040000,2,2
5,-0.95,1.0,-2.0,0.0,-21.0,-0.02,0.0,1.0,2.0,2.0,-1.0,0.0,0.0,0.0,1.0,-0.04,-0.13,-1.99,-19.0,-59.0,38.0,-1.0,0.20,0.0,-0.04,5.0,0.0,0.0,-4.0,-0.28,4.0,0.0,-4.0,0.0,-5.0,-6.0,-0.26,1.0,0.05,0.0,-0.25,-1.0,-4.0,gent-vs-royal-antwerp,Half-Time,https://www.fotmob.com/match/3916965/matchfact...,https://www.oddschecker.com/football/belgium/j...,"{'Draw': 3.126923076923077, 'Antwerp': 1.81666...","[draw, gent, antwerp]","[3.126923076923077, 4.5, 1.8166666666666669]",3.126923,4.500000,1.816667,2,2
6,-0.33,-3.0,1.0,0.0,-104.0,-0.15,-1.0,3.0,-3.0,0.0,0.0,-3.0,0.0,1.0,-4.0,0.31,-0.64,0.71,-97.0,-51.0,-53.0,3.0,-0.09,2.0,0.29,-1.0,0.0,0.0,2.0,-0.06,-1.0,3.0,6.0,1.0,7.0,3.0,0.10,4.0,0.25,-1.0,0.11,1.0,-22.0,randers-fc-vs-aalborg-bk,Half-Time,https://www.fotmob.com/match/3887414/matchfact...,https://www.oddschecker.com/foo